In [1]:
import cv2
import numpy as np
import dlib
from mtcnn import MTCNN
from sklearn.neighbors import KNeighborsClassifier
import os
import joblib

In [3]:
detector = MTCNN()
shape_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_rec_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [4]:
# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)

In [5]:
known_embeddings = []
known_labels = []

In [6]:
images_path = 'Images'
for label in os.listdir(images_path):
    person_path = os.path.join(images_path, label)
    if not os.path.isdir(person_path):
        continue

    for image_name in os.listdir(person_path):
        if not image_name.lower().endswith('.png'):
            continue

        image_path = os.path.join(person_path, image_name)
        image = cv2.imread(image_path)
        faces = detector.detect_faces(image)

        for face in faces:
            x, y, w, h = face['box']
            x, y = max(0, x), max(0, y)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            rect = dlib.rectangle(x, y, x + w, y + h)
            shape = shape_predictor(gray, rect)
            embedding = np.array(face_rec_model.compute_face_descriptor(image, shape))

            known_embeddings.append(embedding)
            known_labels.append(label)

In [7]:
# Train and save the KNN classifier
if known_embeddings:
    knn.fit(known_embeddings, known_labels)
    joblib.dump(knn, 'knn_model.pkl')
    print("KNN model trained and saved as 'knn_model.pkl'.")
else:
    print("No embeddings found. Model not trained.")

KNN model trained and saved as 'knn_model.pkl'.
